# Tradução EN->PT

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
plt.style.use('default')
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import NMF, PCA
from sklearn.cluster import KMeans
from sklearn.manifold import Isomap, TSNE
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras.layers import Input, Dense, TimeDistributed, Embedding, GlobalAveragePooling1D, Flatten, SimpleRNN, GRU, Dropout, LSTM, Bidirectional
from tensorflow.keras.utils import plot_model
from keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report
import os
import re

# Pre-processar dados

In [ ]:
max_lines = 500
lines_pt = []
with open('./datasets/en_pt.pt', 'r+') as f:
    curr_line = 0
    for line in f:
        curr_line = curr_line +1
        lines_pt.append(line)
        if curr_line >= max_lines:
            break

lines_en = []
with open('./datasets/en_pt.en', 'r+') as f:
    curr_line = 0
    for line in f:
        curr_line = curr_line +1
        lines_en.append(line)
        if curr_line >= max_lines:
            break


In [ ]:
print(lines_pt[0])

In [ ]:
# Tokenizar linhas - pt & en

# Entradas serao em portugues!
from keras.preprocessing.text import Tokenizer
from keras.utils.np_utils import to_categorical
from keras.preprocessing.sequence import pad_sequences

vocab_size_pt = 2000
tokenizer_pt = Tokenizer(num_words=vocab_size_pt)
tokenizer_pt.fit_on_texts(lines_pt)
sequences_pt = tokenizer_pt.texts_to_sequences(lines_pt)
sequences_pt = pad_sequences(sequences_pt, maxlen=100)

# Saidas serao em ingles / one-hot encooding
vocab_size_en = 2000
tokenizer_en = Tokenizer(num_words=vocab_size_en)
tokenizer_en.fit_on_texts(lines_en)
sequences_en = tokenizer_en.texts_to_sequences(lines_en)
sequences_en = pad_sequences(sequences_en, maxlen=100)



In [ ]:
X = np.array(sequences_pt)
print(X.shape)

In [ ]:
y_ = np.array(sequences_en)
y_ = y_.reshape(y_.shape[0]*y_.shape[1],1)

onehot_encoder = OneHotEncoder(sparse=False)
y = onehot_encoder.fit_transform(y_)
y = y.reshape(max_lines, 100, vocab_size_en)
print(y.shape)

# Teste de sanidade!
_i = np.argmax(y[15,91,:])
_j = sequences_en[15,91]
print(_i, _j)

# Rede neural tipo encoder-decoder

In [ ]:
def rede_neural_traducao(input_dims, vocab_size, dim_latente, dim_out):
  input_layer = Input(shape=(input_dims,))
  x = input_layer
  x = Embedding(vocab_size, dim_latente, name='projecao')(x)
  x,state_h, state_c = LSTM(512, return_sequences=True, return_state=True)(x)

  x = LSTM(512, return_sequences=True) (x, initial_state=[state_h, state_c])
  y = Dense(dim_out, activation='softmax', name='decisor')(x)
  return Model(input_layer, y)

rede_neural = rede_neural_traducao(100, vocab_size_pt, 100, vocab_size_en)
rede_neural.compile(optimizer='adam', loss='categorical_crossentropy')
plot_model(rede_neural, show_shapes=True, show_layer_activations=True)

In [ ]:
#es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10, restore_best_weights=True)
history = rede_neural.fit(X, y, epochs=700)#, validation_split=0.2, callbacks=es)

In [ ]:
plt.figure(figsize=(7,2))
plt.plot(history.history['loss'], label='loss')
#plt.plot(history.history['val_loss'], label='val_loss')
plt.xlabel('Épocas')
#plt.semilogy()
plt.ylabel('Categorical Cross-Entropy')
plt.legend()
plt.show()

## Teste

In [ ]:
frase_de_teste=lines_pt[2]
frase_de_teste="este ruido leva a resultados ruins"
tokens = tokenizer_pt.texts_to_sequences([frase_de_teste])
tokens = pad_sequences(tokens, maxlen=100)
tokens = np.array(tokens)
print(tokens.shape)

In [ ]:
y_predict = rede_neural.predict(tokens)
y_sequence = np.argmax(y_predict, axis=2)
print(y_sequence)

In [ ]:
y_text = tokenizer_en.sequences_to_texts(y_sequence)
print(y_text)

In [ ]:
lines_en[2]